# Scopus Title Data

In [1]:
import gzip
import os
import re

import pandas

from utilities import format_issn

In [2]:
# Load dataset
path = os.path.join('download', 'title_list.xlsx')
source_df = pandas.read_excel(path, sheetname='Scopus Sources October 2016')
conf_96_df = pandas.read_excel(path, sheetname='Conf. Proceedings post-1995')
conf_95_df = pandas.read_excel(path, sheetname='Conf. Proceedings pre-1996')
code_df = pandas.read_excel(path, sheetname='ASJC Code list ')

In [3]:
# Rename columns on dataframes
renamer = {
    'Sourcerecord id ': 'scopus_id',
    'All Science Classification Codes (ASJC)': 'asjc_codes',
    'ASJC code': 'asjc_codes',
    'Print-ISSN': 'issn_print',
    'E-ISSN': 'issn_electronic',
    'ISSN': 'issn_unspecified',
}

for df in source_df, conf_96_df, conf_95_df:
    df.rename(columns=renamer, inplace=True)

## Titles

In [4]:
# Extract title names
rows = list()
for df in source_df, conf_96_df, conf_95_df:
    rows.extend(zip(df.scopus_id, df.iloc[:, 1]))

rows.sort()
title_df = pandas.DataFrame(rows, columns=['scopus_id', 'title_name'])

In [5]:
path = os.path.join('data', 'titles.tsv')
title_df.to_csv(path, index=False, sep='\t')

In [6]:
title_df.head(2)

,scopus_id,title_name
0,12000,Journal of Technology in Counseling
1,12001,Journal of the Experimental Analysis of Behavior


## ISSN

In [7]:
value_vars = ['issn_print', 'issn_electronic', 'issn_unspecified']
issn_dfs = list()
for df in source_df, conf_96_df, conf_95_df:
    df = pandas.melt(df, id_vars='scopus_id', value_vars=value_vars, var_name='issn_type', value_name='issn')
    df.issn = df.issn.map(format_issn)
    df.issn_type = df.issn_type.map(lambda x: x.split('_')[1])
    issn_dfs.append(df.dropna())
issn_df = (
    pandas.concat(issn_dfs)
    .dropna()
    .sort_values(['scopus_id', 'issn_type', 'issn'])
    .drop_duplicates(['scopus_id', 'issn'])
)

In [8]:
len(issn_df)

45380

In [9]:
issn_df.head(3)

,scopus_id,issn_type,issn
21137,12000,print,1527-6228
21353,12001,print,0022-5002
57251,12002,electronic,1520-6696


In [10]:
path = os.path.join('data', 'issn.tsv')
issn_df.to_csv(path, sep='\t', index=False)

## ASJC Codes

All Science Journal Classification (ASJC) Codes

In [11]:
# Extract ASJC code descriptions
code_df = code_df.rename(columns={'Code': 'asjc_code', 'Description': 'asjc_description'})
code_df = code_df[['asjc_code', 'asjc_description']]

In [12]:
path = os.path.join('data', 'asjc-codes.tsv')
code_df.to_csv(path, index=False, sep='\t')

In [13]:
code_df.head(2)

,asjc_code,asjc_description
0,1000,Multidisciplinary
1,1100,Agricultural and Biological Sciences(all)


In [14]:
# Extract title to code mapping
rows = list()
for df in source_df, conf_96_df, conf_95_df:
    for scopus_id, codes in zip(df.scopus_id, df.asjc_codes):
        for code in re.split(r'[,;] *', str(codes)):
            code = code.strip()
            try:
                code = int(code)
            except ValueError:
                continue
            rows.append((scopus_id, code))

rows.sort()
title_codes_df = pandas.DataFrame(rows, columns=['scopus_id', 'asjc_code'])
title_codes_df = title_codes_df.drop_duplicates()

In [15]:
path = os.path.join('data', 'titles-asjc-codes.tsv')
title_codes_df.to_csv(path, index=False, sep='\t')

In [16]:
title_codes_df.head(2)

,scopus_id,asjc_code
0,12000,1705
1,12000,3304


In [17]:
# Extract title to subject area mapping
subject_df = title_codes_df.copy()
subject_df.asjc_code = 100 * (subject_df.asjc_code // 100)
subject_df = subject_df.drop_duplicates()
subject_df = subject_df.merge(code_df)
subject_df.asjc_description = subject_df.asjc_description.map(lambda x: re.sub(r'\(all\)$', '', x))
subject_df = subject_df.sort_values(['scopus_id', 'asjc_code'])

In [18]:
path = os.path.join('data', 'subject-areas.tsv')
subject_df.to_csv(path, index=False, sep='\t')

In [19]:
subject_df.head(2)

,scopus_id,asjc_code,asjc_description
0,12000,1700,Computer Science
13018,12000,3300,Social Sciences


## Title attributes

In [20]:
# Extract title attributes for non-conference-proceedings
renamer = {
    'Open Acces status, i.e., registered in DOAJ and/or ROAD. Status March 2016\n': 'open_access',
    'Active or Inactive': 'active',
    'Source Type': 'source_type',
    'Publisher imprints grouped to main Publisher': 'main_publisher',
    "Publisher's Country ": 'publisher_country',
}
attribute_df = source_df.copy()
attribute_df = attribute_df.rename(columns=renamer)
attribute_df = attribute_df[['scopus_id'] + list(renamer.values())]
attribute_df.active = (attribute_df.active == 'Active').astype(int)
attribute_df.open_access = (pandas.isnull(attribute_df.open_access)).astype(int)
attribute_df = attribute_df.sort_values('scopus_id')
# Enforce column order for diff viewing
columns = ['scopus_id', 'active', 'open_access', 'main_publisher', 'source_type', 'publisher_country']
attribute_df = attribute_df[columns]

In [21]:
path = os.path.join('data', 'title-attributes.tsv')
attribute_df.to_csv(path, index=False, sep='\t')

In [22]:
attribute_df.head(2)

,scopus_id,active,open_access,main_publisher,source_type,publisher_country
21137,12000,0,1,Columbus State University,Journal,United States
21353,12001,1,1,Society for the Experimental Analysis of Behav...,Journal,United States


In [23]:
# Top level subject areas
rows = []
tl_cols = list(source_df.columns[source_df.columns.str.startswith('Top level:')])
for i, series in source_df[['scopus_id'] + tl_cols].iterrows():
    scopus_id = series.scopus_id
    for value in series.ix[1:].dropna():
        rows.append((scopus_id, value))
rows.sort()
top_df = pandas.DataFrame(rows, columns=['scopus_id', 'top_level_subject'])

In [24]:
path = os.path.join('data', 'title-top-levels.tsv')
top_df.to_csv(path, index=False, sep='\t')

In [25]:
top_df.head(2)

,scopus_id,top_level_subject
0,12000,Physical Sciences
1,12000,Social Sciences
